# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joinning the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
   

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# checking the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster


local_machine =['127.0.0.1']

try:
    cluster = Cluster(local_machine)

# To establish connection  
    session = cluster.connect()
    print('Connection Sucessful')
    
except Exception as e:
    print(f'Connection Error {e}')

Connection Sucessful


#### Creating a Keyspace

In [6]:
 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_b
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting a Keyspace

In [7]:
 
try:
    session.set_keyspace('project_b')
    print('Setup complete')
except Exception as e:
    print(e)


Setup complete


## Creating queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Question 1

**Query 1:**  This query returns the artist, song title and song's length in the music app history that was heard during \
sessionId = 338, and itemInSession = 4

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_by_session"
query = query + "(sessionId int,  itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
session.execute(query)
    
print("Table created")                   

Table created


In [9]:
 
file = 'event_datafile_new.csv'
 #extracting the data from the columns
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    
    
        query = "INSERT INTO songs_by_session (sessionId, itemInSession, artist, song, length )"
        query = query + " VALUES (%s, %s, %s, %s, %s)" 
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ) )
    
    
    
print("Data inserted Successfully")
        
        

Data inserted Successfully


In [10]:

query = "SELECT artist, song, length FROM songs_by_session WHERE sessionId = 338 AND itemInSession =4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
songs_by_session = PrettyTable()    
songs_by_session.field_names = ["artist", "song", "length"]

for row in rows:
    songs_by_session.add_row([row.artist, row.song, row.length])
    print(songs_by_session) 

+-----------+---------------------------------+--------------------+
|   artist  |               song              |       length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


## Question 2

**Query 2:** This query returns the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [11]:

query = "CREATE TABLE IF NOT EXISTS song_info_by_userid"
query = query + "(userId int, sessionId int,  itemInSession int, artist text, song text, first_name text, last_name text,\
                  PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query)
    
print("Table created")                      

Table created


In [12]:
 
file = 'event_datafile_new.csv'
 #extracting the data from the columns
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    
         
        query = "INSERT INTO song_info_by_userid (userId, sessionId, itemInSession, artist, song, first_name, last_name )"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)" 
          
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4] ) )
        
    
print("Data inserted Successfully")
        
        

Data inserted Successfully


In [13]:
 
query = "SELECT artist, song, first_name, last_name FROM song_info_by_userid WHERE userId = 10 AND sessionId =182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
song_info_by_userid = PrettyTable()
song_info_by_userid.field_names = ["artist", "song", "first_name", "last_name"]    


for row in rows:
    song_info_by_userid.add_row([row.artist, row.song, row.first_name, row.last_name]) 
    
    
print(song_info_by_userid)

+-------------------+------------------------------------------------------+------------+-----------+
|       artist      |                         song                         | first_name | last_name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


## Question 3

**Query 3:** this query returms every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
 
query = "CREATE TABLE IF NOT EXISTS user_by_song_title"
query = query + "(song text, userId int, first_name text, last_name text, PRIMARY KEY (song, userId))"
session.execute(query)
    
print("Table created")    
                    

Table created


In [15]:
file = 'event_datafile_new.csv'
 #extracting the data from the columns
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    
         
        query = "INSERT INTO user_by_song_title (song, userId, first_name, last_name )"
        query = query + " VALUES (%s, %s, %s, %s)" 
       
        session.execute(query, (line[9], int(line[10]), line[1], line[4] ) )
    
    
    
print("Data inserted Successfully")
        
        

Data inserted Successfully


In [16]:
 
query = "SELECT first_name, last_name FROM user_by_song_title WHERE song ='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
user_by_song_title = PrettyTable()
user_by_song_title.field_names = ["first_name", "last_name"]    

    
for row in rows:
    user_by_song_title.add_row([row.first_name, row.last_name]) 
    
    
print(user_by_song_title)

+------------+-----------+
| first_name | last_name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Droping all the tables 

In [17]:
query = "SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'project_b'"

all_tables = session.execute(query)

In [18]:
for table in all_tables:
    table_name = table[0]
    session.execute("DROP TABLE " + table_name) 

### Closing the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()